In [10]:
import yfinance as yf
import pandas as pd
import ta

In [27]:
df = yf.download('MSFT', start = '2003-01-01')

[*********************100%%**********************]  1 of 1 completed


In [28]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2003-01-02,26.150000,26.875000,25.855000,26.860001,16.652229,67025200
2003-01-03,26.795000,26.900000,26.440001,26.895000,16.673920,55546800
2003-01-06,27.010000,27.615000,26.900000,27.385000,16.977707,59456200
2003-01-07,27.459999,28.004999,27.340000,27.900000,17.296991,80517400
2003-01-08,27.684999,27.775000,27.055000,27.120001,16.813419,64566000
...,...,...,...,...,...,...
2023-12-18,369.450012,373.000000,368.679993,372.649994,372.649994,21802900
2023-12-19,371.489990,373.260010,369.839996,373.260010,373.260010,20603700
2023-12-20,375.000000,376.029999,370.529999,370.619995,370.619995,26316700


In [42]:
df['SMA'] = df.Close.rolling(200).mean() #ma200
df['rsi'] = ta.momentum.rsi(df.Close,window =10) #RSI
df['price'] = df.Open.shift(-1) #Next Day Open Price
df['Buy'] =  (df.Close > df.SMA) & (df.rsi < 30) # Entry Parameters
df['Sell'] = df.rsi > 40 # Exit Parameters

In [43]:
#Exit of Trade, Either RSI above 40 or after 10 days
in_position = False

buys,sells = [],[]
buydates, selldates = [],[]

for index,row in df.iterrows():
    if not in_position:
        if row.Buy:
            buys.append(row.price)
            print("Bought at:",index)
            in_position = True 
            r_count = 0
    if in_position:
        r_count += 1
        if row.Sell:
            sells.append(row.price)
            print("Sold as RSI is above 40",index)
            in_position = False
        elif r_count >= 10:
            sells.append(row.price)
            print("Sold as in trade for 10 days",index)
            in_position = False
            

Bought at: 2003-10-24 00:00:00
Sold as in trade for 10 days 2003-11-06 00:00:00
Bought at: 2005-09-16 00:00:00
Sold as RSI is above 40 2005-09-29 00:00:00
Bought at: 2005-12-16 00:00:00
Sold as in trade for 10 days 2005-12-30 00:00:00
Bought at: 2007-02-08 00:00:00
Sold as RSI is above 40 2007-02-14 00:00:00
Bought at: 2007-02-27 00:00:00
Sold as in trade for 10 days 2007-03-12 00:00:00
Bought at: 2008-01-22 00:00:00
Sold as RSI is above 40 2008-01-24 00:00:00
Bought at: 2010-01-29 00:00:00
Sold as in trade for 10 days 2010-02-11 00:00:00
Bought at: 2010-05-06 00:00:00
Sold as RSI is above 40 2010-05-12 00:00:00
Bought at: 2011-02-22 00:00:00
Sold as in trade for 10 days 2011-03-07 00:00:00
Bought at: 2012-04-10 00:00:00
Sold as RSI is above 40 2012-04-12 00:00:00
Bought at: 2012-05-16 00:00:00
Sold as RSI is above 40 2012-05-29 00:00:00
Bought at: 2012-06-01 00:00:00
Sold as RSI is above 40 2012-06-06 00:00:00
Bought at: 2013-07-19 00:00:00
Sold as in trade for 10 days 2013-08-01 00:0

In [44]:
#Trades Entry and Exit Price
trades = pd.DataFrame([buys,sells], index=['Buys','Sells'])
trades = trades.T
trades

,Buys,Sells
0,26.910000,26.379999
1,26.090000,25.910000
2,26.820000,26.250000
3,29.350000,29.580000
4,27.950001,27.250000
5,31.480000,34.900002
6,28.389999,27.809999
7,28.930000,29.260000
8,26.530001,25.770000
9,30.430000,30.889999


In [45]:
# PnL Row
trades['PnL'] = (trades.Sells- trades.Buys)/ trades.Buys
trades

,Buys,Sells,PnL
0,26.910000,26.379999,-0.019695
1,26.090000,25.910000,-0.006899
2,26.820000,26.250000,-0.021253
3,29.350000,29.580000,0.007836
4,27.950001,27.250000,-0.025045
5,31.480000,34.900002,0.108640
6,28.389999,27.809999,-0.020430
7,28.930000,29.260000,0.011407
8,26.530001,25.770000,-0.028647
9,30.430000,30.889999,0.015117


In [46]:
#Strategy Win Rate
(trades[trades.PnL > 0].shape[0]/ trades.shape[0])*100

57.14285714285714

In [47]:
(trades.PnL + 1).cumprod() - 1

0    -0.019695
1    -0.026459
2    -0.047149
3    -0.039682
4    -0.063733
5     0.037983
6     0.016778
7     0.028376
8    -0.001084
9     0.014016
10   -0.007623
11    0.027745
12    0.027420
13    0.071629
14    0.133001
15    0.110068
16    0.070193
17    0.079748
18    0.131347
19    0.169577
20    0.160137
21    0.195244
22    0.362990
23    0.389765
24    0.400977
25    0.349870
26    0.427250
27    0.466233
Name: PnL, dtype: float64